<a href="https://colab.research.google.com/github/ArnavMehrotra/ArNet/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
%%shell
if [ ! -d ArNet ]; then
  git clone https://github.com/ArnavMehrotra/ArNet
fi

In [90]:
import time
import ctypes
import numpy as np
import os

max_size = 9000

In [91]:
%%shell
cd ArNet/
git pull

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 4.24 KiB | 4.24 MiB/s, done.
From https://github.com/ArnavMehrotra/ArNet
   962e158..1bbeee8  main       -> origin/main
Updating 962e158..1bbeee8
Fast-forward
 pipeline.ipynb | 833 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 833 insertions(+)
 create mode 100644 pipeline.ipynb


In [92]:
!rm -rf *.so
so_name = f"ops_{int(time.time())}.so"
!nvcc -Xcompiler -fPIC -shared -gencode arch=compute_80,code=sm_80 -o {so_name} ArNet/launch.cu ArNet/kernels.cu
lib = ctypes.CDLL(f"./{so_name}")

In [93]:
def gemmInt(a: np.array, b: np.array) -> np.array:
  j, k = a.shape
  m, n = b.shape

  if(m != k):
    print("matrix dimensions do not match")
    return

  N = j * n
  op1 = np.array(a, dtype=np.int32)
  op2 = np.array(b, dtype=np.int32)

  out = np.zeros(N, dtype=np.int32)

  lib.launchMultInt.argtypes =  [ctypes.POINTER(ctypes.c_int),
                              ctypes.POINTER(ctypes.c_int),
                              ctypes.POINTER(ctypes.c_int),
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int]

  a_ptr = op1.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
  b_ptr = op2.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_int))

  lib.launchMultInt(a_ptr, b_ptr, c_ptr, j, k, m, n)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, n)

  return c_np

In [94]:
def gemm(a: np.array, b: np.array) -> np.array:

  if(a.dtype != np.float32 or b.dtype != np.float32):
    print("data type must be float32")
  j, k = a.shape
  m, n = b.shape

  if(m != k):
    print("matrix dimensions do not match")
    return

  N = j * n

  out = np.zeros(N, dtype=np.float32)

  lib.launchMult.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchMult(a_ptr, b_ptr, c_ptr, j, k, m, n)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, n)

  return c_np

In [95]:
def gemm2(a: np.array, b: np.array) -> np.array:

  lib = ctypes.CDLL(os.environ["LIB"])

  print(os.environ["LIB"])
  if(a.dtype != np.float32 or b.dtype != np.float32):
    print("data type must be float32")
  j, k = a.shape
  m, n = b.shape

  if(m != k):
    print("matrix dimensions do not match")
    return

  N = j * n

  out = np.zeros(N, dtype=np.float32)

  lib.launchMult2.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int,
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchMult2(a_ptr, b_ptr, c_ptr, j, k, m, n)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, n)

  return c_np

In [96]:
def biasAdd(a: np.array, b: np.array) -> np.array:
  if a.dtype != np.float32 or b.dtype != np.float32:
    print("data type must be float32")

  j, k = a.shape
  n = b.shape[0]

  if k != n:
    print("matrix dimensions do not match")
    return

  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchBiasAdd.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_int,
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchBiasAdd(a_ptr, b_ptr, c_ptr, j, k)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, k)

  return c_np


In [97]:
def scalarAdd(a: np.array, s: float) -> np.array:
  if a.dtype != np.float32:
    print("data type must be float32")

  j, k = a.shape
  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchScalarAdd.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_float,
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchScalarAdd(a_ptr, c_ptr, s, N)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, k)

  return c_np

In [98]:
def matAdd(a: np.array, b: np.array) -> np.array:

  if(a.dtype != np.float32 or b.dtype != np.float32):
    print("data type must be float32")

  j, k = a.shape
  m, n = b.shape

  if m != j or n != k:
    print("matrix dimensions do not match")
    return

  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchAdd.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_int,
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  c_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchAdd(a_ptr, b_ptr, c_ptr, j, k)

  c_np = np.ctypeslib.as_array(c_ptr, (N,)).reshape(j, k)

  return c_np

In [99]:
def relu(a: np.array) -> np.array:

  if(a.dtype != np.float32):
    print("data type must be float32")

  j, k = a.shape

  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchRelu.argtypes =  [ctypes.POINTER(ctypes.c_float),
                              ctypes.POINTER(ctypes.c_float),
                              ctypes.c_int]

  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchRelu(a_ptr, b_ptr, N)

  c_np = np.ctypeslib.as_array(b_ptr, (N,)).reshape(j, k)

  return c_np

In [100]:
def softmax(a: np.array) -> np.array:
  if a.dtype != np.float32:
    print("data type must be float32")

  j, k = a.shape
  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchSoftmax.argtypes =  [ctypes.POINTER(ctypes.c_float),
                                 ctypes.POINTER(ctypes.c_float),
                                 ctypes.c_int,
                                 ctypes.c_int]
  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  b_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchSoftmax(a_ptr, b_ptr, j, k)
  c_np = np.ctypeslib.as_array(b_ptr, (N,)).reshape(j, k)

  return c_np

In [101]:
def gradient(a: np.array, y: np.array):
  if a.dtype != np.float32:
    print("data type must be float32")
    return

  if y.dtype != np.uint32:
    print("label index type must be uint32")
    return

  j, k = a.shape
  N = j * k

  out = np.zeros(N, dtype=np.float32)

  lib.launchGradient.argtypes =  [ctypes.POINTER(ctypes.c_float),
                                  ctypes.POINTER(ctypes.c_uint32),
                                  ctypes.POINTER(ctypes.c_float),
                                  ctypes.c_int,
                                  ctypes.c_int]


  a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
  y_ptr = y.ctypes.data_as(ctypes.POINTER(ctypes.c_uint32))
  b_ptr = out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

  lib.launchGradient(a_ptr, y_ptr, b_ptr, j, k)

  b_np = np.ctypeslib.as_array(b_ptr, (N,)).reshape(j, k)

  return b_np

In [102]:
def test_gemm():
  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)
  m = k
  n = np.random.randint(2, max_size)

  hi = np.random.rand(j, k).astype(np.float32) * 100
  hello = np.random.rand(m, n).astype(np.float32) * 100

  start = time.time()
  correct = hi @ hello
  end = time.time()

  print(f"Total time for numpy: {(end - start)*1000:.3f} ms")

  start = time.time()
  test = gemm(hi, hello)
  end = time.time()

  print(f"Total time for young arn: {(end - start)*1000:.3f} ms")

  start = time.time()
  test2 = gemm2(hi, hello)
  end = time.time()

  print(f"Total time for young arn (optimized): {(end - start)*1000:.3f} ms")

  print(f"input 1: {j}x{k} matrix")
  print(f"input 2: {m}x{n} matrix")
  print(f"output: {j}x{n} matrix")


  good = np.allclose(correct, test, rtol=1e-3, atol=1e-3) and np.allclose(correct, test2, rtol=1e-3, atol=1e-3)

  return good

In [103]:
def test_biasAdd():
  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)

  a = np.random.rand(j, k).astype(np.float32)
  b = np.random.rand(k).astype(np.float32)

  correct = a + b
  test = biasAdd(a, b)
  good = np.allclose(correct, test, rtol=1e-3, atol=1e-3)

  print(f"input 1: {j}x{k} matrix")
  print(f"input 2: 1x{k} matrix")
  print(f"output: {j}x{k} matrix")

  return good

In [104]:
def test_scalarAdd():

  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)

  a = np.random.rand(j, k).astype(np.float32)
  s = np.random.rand()

  correct = a + s
  test = scalarAdd(a, s)
  good = np.allclose(correct, test, rtol=1e-3, atol=1e-3)


  return good

In [105]:
def test_matAdd():

  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)

  a = np.random.rand(j, k).astype(np.float32)
  b = np.random.rand(j, k).astype(np.float32)

  correct = a + b
  test = matAdd(a, b)

  good = np.allclose(correct, test, rtol=1e-3, atol=1e-3)

  print(f"input 1: {j}x{k} matrix")
  print(f"input 2: {j}x{k} matrix")
  print(f"output: {j}x{k} matrix")

  return good

In [106]:
def test_relu():

  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)

  a = np.random.randn(j, k).astype(np.float32)

  correct = np.maximum(a, 0)
  test = relu(a)

  good = np.allclose(correct, test, rtol=1e-3, atol=1e-3)

  print(f"input: {j}x{k} matrix")

  return good

In [107]:
def numpy_softmax(Z):
    Z_stable = Z - np.max(Z, axis=1, keepdims=True)
    exp_Z = np.exp(Z_stable)
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

def test_softmax():
  sz = max_size

  j = np.random.randint(2, sz)
  k = np.random.randint(2, sz)

  a = np.random.randn(j, k).astype(np.float32) * 100

  test = softmax(a)

  check = numpy_softmax(a)

  good = np.allclose(test, check, rtol=1e-3, atol=1e-3)

  return good

def test_gradient():
  j = np.random.randint(2, max_size)
  k = np.random.randint(2, max_size)

  a = np.random.rand(j, k).astype(np.float32)
  y = np.random.randint(0, k, size=(j, 1)).astype(np.uint32)

  test = gradient(a, y)

  check = numpy_softmax(a)
  check[np.arange(j), y.squeeze()] -= 1

  good = np.allclose(test, check, rtol=1e-3, atol=1e-3)

  return good

In [108]:
if(not test_biasAdd()): print("biasAdd failed")
if(not test_gemm()): print("gemm failed")
if(not test_matAdd()): print("matAdd failed")
if(not test_scalarAdd()): print("scalarAdd failed")
if(not test_relu()): print("relu failed")
if(not test_softmax()): print("softmax failed")
if(not test_gradient()): print("gradient failed")

input 1: 6372x1610 matrix
input 2: 1x1610 matrix
output: 6372x1610 matrix
Total time for numpy: 160.584 ms
Total time for young arn: 83.243 ms
./ops_1754289890.so
Total time for young arn (optimized): 130.152 ms
input 1: 3602x4781 matrix
input 2: 4781x2557 matrix
output: 3602x2557 matrix
input 1: 5527x6133 matrix
input 2: 5527x6133 matrix
output: 5527x6133 matrix
input: 4583x2566 matrix


In [109]:
# lr = 0.001
# epochs = 100

# J, D, H, K = 64, 32, 16, 4  # batch size, input dim, hidden dim, output classes

# X = np.random.randn(J, D).astype(np.float32)
# Y = np.random.randint(0, K, size=(J, 1)).astype(np.uint32)

# W1 = np.random.randn(D, H).astype(np.float32) * 0.01
# b1 = np.zeros((H,), dtype=np.float32)

# W2 = np.random.randn(H, K).astype(np.float32) * 0.01
# b2 = np.zeros((K,), dtype=np.float32)
# for i in range(epochs):
#   #forward pass
#   M1 = gemm(X, W1)
#   Z1 = biasAdd(M1, b1)
#   A1 = relu(Z1)
#   M2 = gemm(A1, W2)
#   Z2 = biasAdd(M2, b2)
#   A2 = softmax(Z2)
#   probs = A2[np.arange(J), Y.squeeze()]
#   loss = -np.log(probs + 1e-8).mean()

#   print(loss)

#   #backprop
#   dZ2 = gradient(A2, Y)
#   dW2 = gemm(A1.T, dZ2)
#   db2 = sum(dZ2)
#   dA1 = gemm(dZ2, W2.T)
#   dZ1 = dA1 * (Z1 > 0)
#   dW1 = gemm(X.T, dZ1)
#   db1 = sum(dZ1)

#   #update weights
#   W1 -= lr * dW1
#   b1 -= lr * db1

#   W2 -= lr * dW2
#   b2 -= lr * db2